In [45]:
#!pip install reedsolo

In [50]:
# Erforderliche Bibliotheken: Dieses Notebook benötigt keine externen Bibliotheken wie numpy.

from IPython.display import display, Markdown, HTML
import math

# --- Teil 1: Die Werkzeuge - Mathematik im Galois-Körper GF(2^8) ---

display(Markdown("## Teil 1: Galois Field GF(2^8)"))
display(Markdown("""
Normale Arithmetik funktioniert hier nicht. Alle Berechnungen mit Bytes (0-255) müssen wieder ein Ergebnis zwischen 0-255 liefern.
"""))

# Initialize the log and anti-log tables
exp_table = [0] * 256
log_table = [0] * 256
primitive_polynomial = 0b100011101

# Generate the tables
x = 1
for i in range(255):
    exp_table[i] = x
    log_table[x] = i
    x <<= 1
    if x & 0x100:
        x ^= primitive_polynomial

exp_table[255] = exp_table[0]

def gf_multiply(a, b):
    """Multiplies two numbers in the GF(2^8) field."""
    if a == 0 or b == 0:
        return 0
    return exp_table[(log_table[a] + log_table[b]) % 255]

def gf_poly_multiply(p1, p2):
    """Multiplies two polynomials in the GF(2^8) field."""
    result_len = len(p1) + len(p2) - 1
    result = [0] * result_len
    for i in range(len(p1)):
        for j in range(len(p2)):
            result[i + j] ^= gf_multiply(p1[i], p2[j])
    return result

def gf_poly_div_detailed(dividend, divisor, printb = False):
    """
    Performs and displays detailed polynomial division in GF(2^8).
    Returns the remainder.
    """
    result = list(dividend)
    divisor_len = len(divisor)
    if printb:
      display(Markdown("#### Start der Polynomdivision:"))
      display(Markdown(f"**Dividend:** `{result}`\n\n**Divisor:** `{divisor}`"))

    for i in range(len(dividend) - divisor_len + 1):
        coefficient = result[i]

        if printb:
          display(Markdown(f"--- \n### PolyDiv: Schritt {i+1}"))

        if coefficient == 0:
            if printb:
              display(Markdown("Führende Ziffer ist 0, kein Rechenschritt nötig. Wir rücken eins weiter."))
            continue

        term_to_subtract = [0] * i + [gf_multiply(d, coefficient) for d in divisor]
        term_to_subtract.extend([0] * (len(result) - len(term_to_subtract)))

        div_str = "&nbsp;".join(f"{n:03}" for n in result)
        sub_str = "&nbsp;".join(f"{n:03}" for n in term_to_subtract)

        if printb:
          display(Markdown(f"Die vorderste Ziffer ist **{coefficient}**. Wir müssen das **{coefficient}-fache des Divisors** subtrahieren (XOR)."))

        next_result = [a ^ b for a, b in zip(result, term_to_subtract)]
        res_str = "&nbsp;".join(f"{n:03}" for n in next_result)

        html_calculation = f"""
        <div style="font-family: monospace; line-height: 1.6; font-size: 14px;">
          &nbsp;&nbsp;&nbsp;&nbsp;{div_str} (Aktueller Dividend)<br/>
          XOR {sub_str} ({coefficient} * Divisor)<br/>
          ------------------------------------------------------------------<br/>
          = &nbsp;&nbsp;{res_str} (Neuer Zwischenstand)
        </div>
        """
        if printb:
          display(HTML(html_calculation))

        result = next_result

    remainder = result[-(divisor_len - 1):]
    display(Markdown(f"--- \n**Division beendet.**"))
    return remainder

# --- Teil 2: Die Berechnung ---

display(Markdown("## Teil 2: Berechnung für den Text 'HELLO'"))

# REALISTISCHES SZENARIO: Wir wählen eine QR-Code Version und einen Fehler-Level
# Dies bestimmt die exakte Kapazität für Daten und Fehlerkorrektur-Bytes.
# Format: (Total Data Codewords, Total Error Correction Codewords)
QR_CAPACITY_TABLE_V1 = {
    'L': (19, 7),
    'M': (16, 10),
    'Q': (13, 13),
    'H': (9, 17)
}
CHOSEN_LEVEL = 'M' # Wir wählen Level Medium
DATA_CAPACITY, num_error_bytes = QR_CAPACITY_TABLE_V1[CHOSEN_LEVEL]


display(Markdown(f"Wir simulieren einen **QR-Code Version 1 mit Fehler-Level '{CHOSEN_LEVEL}'**."))
display(Markdown(f"Gemäss Standard hat dieser Code Platz für **{DATA_CAPACITY} Daten-Bytes** und benötigt exakt **{num_error_bytes} Fehlerkorrektur-Bytes**."))


# Schritt 1: Daten in Bytes umwandeln
input_string = "MAT2"
message_bytes = [ord(c) for c in input_string]
display(Markdown(f"### Schritt 1: Nachricht in Bytes umwandeln"))
display(Markdown(f"'{input_string}'  => `{message_bytes}`"))

# Schritt 1.5: Füll-Bytes (Padding) hinzufügen
display(Markdown(f"### Schritt 1.5: Füll-Bytes (Padding) hinzufügen"))
display(Markdown(f"Der Code hat Platz für **{DATA_CAPACITY}** Daten-Bytes. Unsere Nachricht hat aber nur **{len(message_bytes)}** Bytes."))
display(Markdown("Der leere Platz wird mit standardisierten, abwechselnden Füll-Bytes (236 und 17) aufgefüllt, bis die Kapazität erreicht ist."))

padding_bytes = []
padding_needed = DATA_CAPACITY - len(message_bytes)
for i in range(padding_needed):
    if i % 2 == 0:
        padding_bytes.append(236) # Standard-Padding-Byte 1
    else:
        padding_bytes.append(17)  # Standard-Padding-Byte 2
display(Markdown(f"Benötigte Füll-Bytes: {padding_needed} => `{padding_bytes}`"))

data_payload = message_bytes + padding_bytes
display(Markdown(f"**Voller Datenblock (Nachricht + Padding):** `{data_payload}`"))

# Schritt 2: Das standardisierte Generatorpolynom
display(Markdown(f"### Schritt 2: Das genormte Generatorpolynom laden"))
display(Markdown(f"Für **{num_error_bytes}** Fehlerkorrektur-Bytes ist folgendes Generatorpolynom im Standard definiert:"))

generator_poly = [1]
for i in range(num_error_bytes):
    generator_poly = gf_poly_multiply(generator_poly, [1, exp_table[i]])
display(Markdown(f"`{generator_poly}`"))

# Schritt 3: Nachrichtenpolynom für die Division vorbereiten
display(Markdown(f"### Schritt 3: Nachricht für die Division vorbereiten"))
display(Markdown(f"Wir hängen {num_error_bytes} Nullen an unseren **vollen Datenblock** an, um Platz für den Rest zu schaffen."))
padded_message_bytes = data_payload + [0] * num_error_bytes
display(Markdown(f"Vorbereitete Nachricht: `{padded_message_bytes}`"))

# Schritt 4: Die Polynomdivision durchführen
display(Markdown(f"### Schritt 4: Die Polynomdivision"))
display(Markdown("Jetzt teilen wir den vorbereiteten, vollen Datenblock durch das Generatorpolynom."))
error_correction_bytes = gf_poly_div_detailed(padded_message_bytes, generator_poly)

# Schritt 5: Ergebnis
display(Markdown(f"### Schritt 5: Das Ergebnis"))
display(Markdown(f"Der **Rest** der Division ist `{error_correction_bytes}`."))
display(Markdown(f"**Das sind unsere {num_error_bytes} Fehlerkorrektur-Bytes!**"))

final_codeword = data_payload + error_correction_bytes
display(Markdown(f"### Schritt 6: Die finale Nachricht für den QR-Code"))
display(Markdown("Der volle Datenblock und die Fehlerkorrektur-Bytes werden zusammengefügt."))
display(Markdown(f"**Finale Codewörter:** `{final_codeword}`"))


# --- Teil 3: Visualisierung der Daten (Vereinfacht) ---

display(Markdown("## Teil 3: Vom Byte zum Bild (Vereinfachte Visualisierung)"))
display(Markdown("""
Wir wandeln jedes Byte in seine 8-Bit-Darstellung um und zeichnen für jede '1' ein farbiges Kästchen.

**Wichtiger Hinweis:** Dies ist **kein scannbarer QR-Code!** Es ist nur eine simple Darstellung der reinen Daten.
"""))

# Schritt 7: Bytes in einen Bit-String umwandeln
bit_string = "".join(f"{byte:08b}" for byte in final_codeword)
display(Markdown(f"### Schritt 7: Codewörter in Bits umwandeln"))
display(Markdown(f"Die {len(final_codeword)} Bytes werden zu einem langen String aus {len(bit_string)} Bits:"))
display(Markdown(f"<div style='word-wrap:break-word; font-family:monospace; font-size:12px;'>{bit_string}</div>"))


# Schritt 8: Bit-String als Gitter visualisieren
display(Markdown(f"### Schritt 8: Die Bits als Gitter darstellen (Daten, Padding, Fehlerkorrektur)"))

# Add a legend
legend_html = """
<div style="margin-top: 15px; margin-bottom: 5px;">
    <b>Legende:</b>
    <span style="display:inline-flex; align-items:center; margin-left:10px;"><div style="width:15px; height:15px; background-color:black; border:1px solid #ccc; margin-right:5px;"></div> Original-Daten (1)</span>
    <span style="display:inline-flex; align-items:center; margin-left:10px;"><div style="width:15px; height:15px; background-color:silver; border:1px solid #ccc; margin-right:5px;"></div> Füll-Daten (1)</span>
    <span style="display:inline-flex; align-items:center; margin-left:10px;"><div style="width:15px; height:15px; background-color:crimson; border:1px solid #ccc; margin-right:5px;"></div> Fehlerkorrektur-Bit (1)</span>
    <span style="display:inline-flex; align-items:center; margin-left:10px;"><div style="width:15px; height:15px; background-color:white; border:1px solid #ccc; margin-right:5px;"></div> 0-Bit</span>
</div>
"""
display(HTML(legend_html))

# We know the boundaries of each data type
num_message_bits = len(message_bytes) * 8
num_padding_bits = len(padding_bytes) * 8

total_bits = len(bit_string)
cols = 8 # Each row is one byte

html_grid = f"<div style='display: grid; grid-template-columns: repeat({cols}, 20px); gap: 1px; border: 2px solid #333; width: fit-content; margin-top: 10px;'>"
for i, bit in enumerate(bit_string):
    color = "white" # Default for '0' bits
    if bit == '1':
        if i < num_message_bits:
            color = "black"  # Original message
        elif i < num_message_bits + num_padding_bits:
            color = "silver" # Padding bytes
        else:
            color = "crimson" # Error correction bytes

    html_grid += f"<div style='width: 20px; height: 20px; background-color: {color};'></div>"
html_grid += "</div>"

display(HTML(html_grid))


# Schritt-für-Schritt-Berechnung der QR-Code-Fehlerkorrektur (Detailliert)

## Teil 1: Galois Field GF(2^8)


Normale Arithmetik funktioniert hier nicht. Alle Berechnungen mit Bytes (0-255) müssen wieder ein Ergebnis zwischen 0-255 liefern.


## Teil 2: Berechnung für den Text 'HELLO'

Wir simulieren einen **QR-Code Version 1 mit Fehler-Level 'M'**.

...


Wir wandeln jedes Byte in seine 8-Bit-Darstellung um und zeichnen für jede '1' ein farbiges Kästchen.

**Wichtiger Hinweis:** Dies ist **kein scannbarer QR-Code!** Es ist nur eine simple Darstellung der reinen Daten.


### Schritt 7: Codewörter in Bits umwandeln

Die 26 Bytes werden zu einem langen String aus 208 Bits:

<div style='word-wrap:break-word; font-family:monospace; font-size:12px;'>0100110101000001010101000011001011101100000100011110110000010001111011000001000111101100000100011110110000010001111011000001000111011000010001011110011101111100101111001010101001011011111011000011110111110000</div>

### Schritt 8: Die Bits als Gitter darstellen (Daten, Padding, Fehlerkorrektur)

In [58]:
# 1. Fehlererkennung: Existiert ein Fehler in den Daten?
# 2. Fehlerlokalisierung: An welchen Positionen befinden sich Fehler?
# 3. Datenkorrektur: Die Umkehrung der Fehler, um die Originaldaten wiederherzustellen

# 1. Fehlererkennung
correct = final_codeword
print("The correct codeword: ", correct)

damaged = correct.copy()

def damage(index, damagedValue):
    damaged[index] = damagedValue

damage(3, 111)

print("The damaged codeword: ", damaged)

for i, (a, b) in enumerate(zip(correct, damaged)):
    if a != b:
        print(f"Difference at index {i}: {a} (correct) vs {b} (damaged)")

num_error_bytes = 0

if CHOSEN_LEVEL == 'L':
    num_error_bytes = 7
elif CHOSEN_LEVEL == 'M':
    num_error_bytes = 10
elif CHOSEN_LEVEL == 'Q':
    num_error_bytes = 13
elif CHOSEN_LEVEL == 'H':
    num_error_bytes = 17

def gf_mul(x, y, texp_table, tlog_table):
    if x == 0 or y == 0:
        return 0
    return texp_table[(tlog_table[x] + tlog_table[y]) % 255]

def poly_eval(poly, x, texp_table, tlog_table):
    result = 0
    for coeff in poly:
        result = gf_mul(result, x, texp_table, tlog_table) ^ coeff
    return result

def compute_syndromes(dmged, num_error_bytes):
    syndromes = []
    poly = list(dmged)
    for i in range(num_error_bytes):
        x = exp_table[i]
        val = poly_eval(poly, x, exp_table, log_table)
        syndromes.append(val)
    return syndromes

syndromes = compute_syndromes(damaged, num_error_bytes)
print("Syndrome:", syndromes)


The correct codeword:  [77, 65, 84, 50, 236, 17, 236, 17, 236, 17, 236, 17, 236, 17, 236, 17, 216, 69, 231, 124, 188, 170, 91, 236, 61, 240]
The damaged codeword:  [77, 65, 84, 111, 236, 17, 236, 17, 236, 17, 236, 17, 236, 17, 236, 17, 216, 69, 231, 124, 188, 170, 91, 236, 61, 240]
Difference at index 3: 50 (correct) vs 111 (damaged)
Syndrome: [93, 120, 17, 236, 168, 63, 165, 89, 247, 142]


In [59]:
# 2. Fehlerlokalisierung
def berlekamp_massey(syndromes, exp_table, log_table):
    n = len(syndromes)
    C = [1] + [0] * n
    B = [1] + [0] * n
    L = 0
    m = 1
    b = 1
    for n_idx in range(n):
        d = syndromes[n_idx]
        for i in range(1, L + 1):
            d ^= gf_mul(C[i], syndromes[n_idx - i], exp_table, log_table)
        if d == 0:
            m += 1
        else:
            T = C[:]
            factor = gf_mul(d, exp_table[255 - log_table[b]], exp_table, log_table)
            for i in range(m, n + 1):
                if B[i - m] != 0:
                    C[i] ^= gf_mul(factor, B[i - m], exp_table, log_table)
            if 2 * L <= n_idx:
                L = n_idx + 1 - L
                B = T
                b = d
                m = 1
            else:
                m += 1
    return C[:L+1]

# Chien-Suche zur Fehlerpositionsbestimmung
def chien_search(locator_poly, code_length, texp_table, tlog_table):
    error_positions = []
    for i in range(code_length):
        x_inv = texp_table[(255 - i) % 255]
        result = 0
        for j, coef in enumerate(locator_poly):
            result ^= gf_mul(coef, texp_table[(tlog_table[x_inv] * j) % 255], texp_table, tlog_table)
        if result == 0:
            error_positions.append(i)
    return error_positions

locator_poly = berlekamp_massey(syndromes, exp_table, log_table)
print("Fehler-Lokalisierer-Polynom Λ(x):", locator_poly)

error_positions = chien_search(locator_poly, len(damaged), exp_table, log_table)
print("Fehlerhafte Koeffizienten (Potenzen): ", error_positions)

error_indices = []

for position in error_positions:
    index = len(damaged) - 1 - position
    error_indices.append(index)

error_indices.sort()

print("Fehlerhafte Indizes: ", error_indices)

Fehler-Lokalisierer-Polynom Λ(x): [1, 234]
Fehlerhafte Koeffizienten (Potenzen):  [22]
Fehlerhafte Indizes:  [3]


In [57]:
# 3. Fehlerkorrektur

# 1. Fehler-Auswerter-Polynom Ω(x) (Omega) berechnen
syndromes_poly = syndromes[:]

num_errors = len(locator_poly) - 1

# Polynom-Multiplikation durchführen UND auf den richtigen Grad kürzen
omega_full = gf_poly_multiply(syndromes_poly, locator_poly)
omega_poly = omega_full[:num_errors]

print("Fehler-Auswerter-Polynom Ω(x):", omega_poly)

# 2. Ableitung von Λ'(x) (Lambda) berechnen
lambda_derivative = []

n = len(locator_poly) - 1
for i in range(n + 1): # Korrigiert auf n+1, um alle Koeffizienten zu berücksichtigen
    if i % 2 == 1: # Koeffizienten ungerader Potenzen
        lambda_derivative.append(locator_poly[n-i])
# In unserem Fall (Λ(x) = x + 201) ist die Ableitung Λ'(x) = 1.

print("Ableitung Λ'(x) (korrigiert):", lambda_derivative)

# 3. Fehlerwerte (Magnituden) für jeden Index berechnen
error_magnitudes = []
codeword_length = len(damaged)

for index in error_indices:
    position = index
    field_position = codeword_length - 1 - position  # Chien-Suchposition
    x_inv = exp_table[255 - field_position]

    omega_val = poly_eval(omega_poly, x_inv, exp_table, log_table)
    lambda_prime_val = poly_eval(lambda_derivative, x_inv, exp_table, log_table)
    lambda_prime_inv = exp_table[255 - log_table[lambda_prime_val]]

    magnitude = gf_multiply(omega_val, lambda_prime_inv)
    error_magnitudes.append(magnitude)


# 4. Fehler korrigieren
corrected_codeword = damaged.copy()
for i in range(len(error_indices)):
    index = error_indices[i]
    position = index  # tatsächlicher Byteindex
    field_position = codeword_length - 1 - position
    x_inv = exp_table[255 - field_position]
    omega_val = poly_eval(omega_poly, x_inv, exp_table, log_table)
    lambda_prime_val = poly_eval(lambda_derivative, x_inv, exp_table, log_table)

    magnitude = error_magnitudes[i]
    before = damaged[index]
    corrected_codeword[index] ^= magnitude
    after = corrected_codeword[index]

    print(f"\nFehler an Byte-Index {index}:")
    print(f"  → Feldposition (für x⁻¹): {field_position}")
    print(f"  → x_inv (α^−pos): {x_inv}")
    print(f"  → Ω(x_inv): {omega_val}")
    print(f"  → Λ'(x_inv): {lambda_prime_val}")
    print(f"  → Fehlerwert = {magnitude}, XOR: {before} ^ {magnitude} = {after}")

# Überprüfung:
print("\nOriginales, korrektes Codewort: ", correct)
print("Beschädigtes Codewort:        : ", damaged)
print("Wiederhergestelltes Codewort  : ", corrected_codeword)



if corrected_codeword == correct:
    print("\nKorrektur ERFOLGREICH!")
else:
    print("\nKorrektur FEHLGESCHLAGEN!")

Fehler-Auswerter-Polynom Ω(x): [93]
Ableitung Λ'(x) (korrigiert): [1]

Fehler an Byte-Index 3:
  → Feldposition (für x⁻¹): 22
  → x_inv (α^−pos): 243
  → Ω(x_inv): 93
  → Λ'(x_inv): 1
  → Fehlerwert = 93, XOR: 111 ^ 93 = 50

Originales, korrektes Codewort:  [77, 65, 84, 50, 236, 17, 236, 17, 236, 17, 236, 17, 236, 17, 236, 17, 216, 69, 231, 124, 188, 170, 91, 236, 61, 240]
Beschädigtes Codewort:        :  [77, 65, 84, 111, 236, 17, 236, 17, 236, 17, 236, 17, 236, 17, 236, 17, 216, 69, 231, 124, 188, 170, 91, 236, 61, 240]
Wiederhergestelltes Codewort  :  [77, 65, 84, 50, 236, 17, 236, 17, 236, 17, 236, 17, 236, 17, 236, 17, 216, 69, 231, 124, 188, 170, 91, 236, 61, 240]

Korrektur ERFOLGREICH!
